# 3F7 Lab: CamZIP

## Tree data structures

Import all functions from the package trees where we put together a number of tools for handling trees in the 3F7 lab.

In [1]:
from trees import *
t = [-1, 0, 0, 0, 1, 3, 4, 4, 3]
print(tree2code(t))

{'0': [1], '1': [2, 0], '2': [0, 0, 0], '3': [0, 0, 1], '4': [2, 1]}


Now define a simple tree (play around with this command and construct more complicated trees....)

In [2]:
t = [-1,0,1,1,0]

The following command will print a string that can be copy-pasted into a tree visualising website like [phylo.io](https://phylo.io) (don't forget to add a new line at the end of the string after cutting and pasting)

In [3]:
print(tree2newick(t))
print('Cut and paste the string on the previous line and add a "new line" at the end of the string.')

(( ,),)
Cut and paste the string on the previous line and add a "new line" at the end of the string.


You can also add labels to the nodes in the `tree2newick` command.

In [4]:
print(tree2newick(t,['root', 'child 0', 'grandchild 0', 'grandchild 1', 'child 1']))

((grandchild 0,grandchild 1)child 0,child 1)root


If there are less labels than nodes, the labels will be interpreted "leaves first"

In [5]:
print(tree2newick(t,['symbol 0','symbol 1', 'symbol 2']))

((symbol 0,symbol 1)4,symbol 2)3


The following command converts a variable-length code described by a tree to a code table format.

In [6]:
print(tree2code(t))

{'0': [0, 0], '1': [0, 1], '2': [1]}


Verify that the inverse function can recover the tree. 

In [7]:
print(code2tree(tree2code(t)))

[-1, 0, 1, 1, 0]


But the following may happen as well. Can you explain why?

In [8]:
print(code2tree(tree2code([3,3,4,4,-1])))

[-1, 0, 1, 1, 0]


In [9]:
print(tree2newick([3,3,4,4,-1], ['grandchild 0', 'grandchild 1', 'child 0', 'child 1', 'root']))

(child 0,(grandchild 0,grandchild 1)child 1)root


Similarly but far more problematic is the following inversion. The resulting assignment of codeword to symbols is fundamentally different from the original and would result in wrong decoding.

In [10]:
print(tree2code(code2tree({'0':[1], '1':[0,1], '2':[0,0,1], '3':[0,0,0]})))

{'0': [0], '1': [1, 0], '2': [1, 1, 0], '3': [1, 1, 1]}


These problems are all solved when using the extended tree format.

In [11]:
xt = tree2xtree([3,3,4,4,-1], ['a', 'b', 'c'])
print(xt)

[[3, [], 'a'], [3, [], 'b'], [4, [], 'c'], [4, [0, 1], '3'], [-1, [2, 3], '4']]


In [12]:
print(xtree2code(code2xtree({'0':[1], '1':[0,1], '2':[0,0,1], '3':[0,0,0]})))

{'0': [1], '1': [0, 1], '2': [0, 0, 1], '3': [0, 0, 0]}


## Testing your Shannon-Fano Code

This next section can only be completed once you have a working Shannon-Fano function `shannon_fano()`

In [13]:
from vl_codes import shannon_fano
from random import random
p = [random() for k in range(16)]
p = dict([(chr(k+ord('a')),p[k]/sum(p)) for k in range(len(p))])
print(f'Probability distribution: {p}\n')
c = shannon_fano(p)
print(f'Codebook: {c}\n')
xt = code2xtree(c)
print(f'Cut and paste for phylo.io: {xtree2newick(xt)}')

Probability distribution: {'a': 0.07196274431078038, 'b': 0.06234136365276155, 'c': 0.12358411854971166, 'd': 0.021793990371190997, 'e': 0.03214096732219923, 'f': 0.12799533888773412, 'g': 0.10746631529621319, 'h': 0.06263177321003886, 'i': 0.12431359386617415, 'j': 0.025494378439643462, 'k': 0.011852471189456792, 'l': 0.03232313054100878, 'm': 0.10607756362666905, 'n': 0.046655582862878305, 'o': 0.03863033036205724, 'p': 0.004736337511482199}

Codebook: {'f': [0, 0, 0], 'i': [0, 0, 1, 0], 'c': [0, 1, 0, 0], 'g': [0, 1, 1, 0], 'm': [0, 1, 1, 1], 'a': [1, 0, 0, 1], 'h': [1, 0, 1, 0], 'b': [1, 0, 1, 1, 1], 'n': [1, 1, 0, 0, 1], 'o': [1, 1, 0, 1, 0], 'l': [1, 1, 0, 1, 1], 'e': [1, 1, 1, 0, 0], 'j': [1, 1, 1, 0, 1, 1], 'd': [1, 1, 1, 1, 0, 1], 'k': [1, 1, 1, 1, 1, 0, 1], 'p': [1, 1, 1, 1, 1, 1, 1, 0]}

Cut and paste for phylo.io: (((f,(i)3)2,((c)5,(g,m)6)4)1,(((a)9,(h,(b)11)10)8,(((n)14,(o,l)15)13,((e,(j)18)17,((d)20,((k)22,((p)24)23)21)19)16)12)7)0


We can upload data from a file, for example `hamlet.txt`, and display the first few lines...

In [14]:
f = open('hamlet.txt', 'r')
hamlet = f.read()
f.close()
print(hamlet[:294])

        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. (KING CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


We now compute the startistics of the file:

In [15]:
from itertools import groupby
frequencies = dict([(key, len(list(group))) for key, group in groupby(sorted(hamlet))])
Nin = sum([frequencies[a] for a in frequencies])
p = dict([(a,frequencies[a]/Nin) for a in frequencies])
print(f'File length: {Nin}')

File length: 207038


We can view the alphabet of symbols used in the file:

In [16]:
print(list(p))
len(p)

['\n', ' ', '!', '&', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|']


67

We are now ready to construct the Shannon-Fano code for this file, and view its tree (cut and paste into [phylo.io](https://phylo.io), don't forget to add a carriage return at the end, click on "Branch Labels/Support" under "Settings", then right-click on the root of the tree and select "expand all". 

In [17]:
c = shannon_fano(p)
print(xtree2newick(code2xtree(c)))

((space,((e,(t)4)3,((o)6,(a,s)7)5)2)1,((((n)11,(h,i)12)10,((r,(carriage return)15)14,((l)17,(d)18)16)13)9,((((u,m)22,(comma,(y)24)23)21,(((w)27,(f)28)26,((c,g)30,(p)31)29)25)20,(((((A)36,(b,T)37)35,((I)39,(E)40)38)34,(((.)43,(L)44)42,((v)46,(k,O)47)45)41)33,((((')51,(H,R)52)50,((N,(S)55)54,((U)57,(M)58)56)53)49,((((semi-colon,colon)62,(D)63)61,((C,G)65,(W,?)66)64)60,(((-,(P)70)69,((!)72,(left square bracket,right square bracket)73)71)68,(((B,F)76,((x,K)78)77)75,(((q)81,(Y,(j)83)82)80,(((Q)86,(Z)87)85,((z,(vertical bar)90)89,((V)92,((left parenthesis,right parenthesis)94,((J)96,((&)98)97)95)93)91)88)84)79)74)67)59)48)32)19)8)0


Now we can actually encode the file `hamlet.txt` using the Shannon-Fano code we constructed.

In [18]:
from vl_codes import vl_encode
hamlet_sf = vl_encode(hamlet, c)
print(f'Length of binary sequence: {len(hamlet_sf)}')

Length of binary sequence: 997542


We have commands to convert a bit sequence into a byte sequence (including a 3 bit prefix that helps us determine the length of the bit sequence):

In [19]:
from vl_codes import bytes2bits, bits2bytes
x = bits2bytes([0,1])
print([format(a, '08b') for a in x])
y = bytes2bits(x)
print(f'The original bits are: {y}')

#print(bits2bytes([0,1,1,0,1,1,0,0,0]))

['01101000']
The original bits are: [0, 1]


We now apply the bits to byte conversion to the compressed text of Hamlet to compute the length of the compressed file.

In [20]:
hamlet_zipped = bits2bytes(hamlet_sf)
Nout = len(hamlet_zipped)
print(f'Length of compressed string: {Nout}')

Length of compressed string: 124694


The compression ratio can be expressed in two ways, unitless or in bits/bytes:

In [21]:
print(f'Compression ratio (rateless): {Nout/Nin}')
print(f'Compression ratio (bits per byte): {8.0*Nout/Nin}')

Compression ratio (rateless): 0.6022759107023832
Compression ratio (bits per byte): 4.818207285619065


The lower bound for compression is the Entropy, measured in bits, that can be computed using an in-line function in Python:

In [22]:
from math import log2
H = lambda pr: -sum([pr[a]*log2(pr[a]) for a in pr])
print(f'Entropy: {H(p)}')

Entropy: 4.4498605005366585


We now proceed to decode the compressed Hamlet sequence

In [23]:
from vl_codes import vl_decode
xt = code2xtree(c)
hamlet_unzipped = vl_decode(hamlet_sf, xt)
print(f'Length of unzipped file: {len(hamlet_unzipped)}')

Length of unzipped file: 207038


We can view the first few lines of the input (note the command `join` that turns the list of strings into one string)

In [24]:
print(''.join(hamlet_unzipped[:294]))

        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. (KING CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


## Compressing and uncompressing files

This is where we put it all together, compressing directly from input to output file. Play around with these commands once you implemented Huffman coding and arithmetic coding. We begin by importing the compression and decompression functions.

In [25]:
from camzip import camzip
from camunzip import camunzip

The next commands define the method to be used and the filename. Modify those when you are trying other methods on various files. 

In [31]:
method = 'arithmetic'
filename = 'hamlet.txt'
camzip(method, filename)

Now we do the actual compression and decompression...

In [32]:
#camzip(method, filename)
camunzip(filename + '.cz' + method[0])

The next few lines perform various statistical measurements and verifies that the decompressed file is identical to the compressed file.

In [33]:
from filecmp import cmp
from os import stat
from json import load
Nin = stat(filename).st_size
print(f'Length of original file: {Nin} bytes')
Nout = stat(filename + '.cz' + method[0]).st_size
print(f'Length of compressed file: {Nout} bytes')
print(f'Compression rate: {8.0*Nout/Nin} bits/byte')
with open(filename + '.czp', 'r') as fp:
    freq = load(fp)
pf = dict([(a, freq[a]/Nin) for a in freq])
print(f'Entropy: {H(pf)} bits per symbol')
if cmp(filename,filename+'.cuz'):
    print('The two files are the same')
else:
    print('The files are different')

Length of original file: 207038 bytes
Length of compressed file: 115162 bytes
Compression rate: 4.449888426279234 bits/byte
Entropy: 4.4498605005366585 bits per symbol
The two files are the same


## Huffman coding

This section will only work once you have a working function `huffman()`. We first repeat the tree construction and visualisation.

In [38]:
from vl_codes import huffman
xt = huffman(p)
print(xtree2newick(xt))

{'\n': 0.029197397591758073, ' ': 0.2731417752210936, '!': 0.0014538323697467627, '&': 2.4150039364564164e-05, "'": 0.0046657876052337965, '(': 7.728012596660533e-05, ')': 7.728012596660533e-05, ',': 0.015852085838899917, '-': 0.0020382633223692153, '.': 0.0062886702505325085, ':': 0.0026661643458478837, ';': 0.0028110645820352688, '?': 0.0020672433696066923, 'A': 0.007732842604533445, 'B': 0.0011688619052449055, 'C': 0.0023425538183627238, 'D': 0.002511604093914673, 'E': 0.00710977158892769, 'F': 0.0011205618265157772, 'G': 0.002130033471954559, 'H': 0.0046078275107588424, 'I': 0.0074140620849211985, 'J': 4.3470070856215496e-05, 'K': 0.0006423910470974068, 'L': 0.005752539376639184, 'M': 0.0030863750307913, 'N': 0.004071696636865518, 'O': 0.005201918479127121, 'P': 0.001477982409111327, 'Q': 0.0004685107636725448, 'R': 0.00434700708562155, 'S': 0.003815706219601138, 'T': 0.007525152265998193, 'U': 0.0032216152512328593, 'V': 0.0001932003149165133, 'W': 0.0021252034640816464, 'Y': 0.00

NameError: Tree with no root or several roots

Observe how the Huffman tree differs from the Shannon-Fano tree. What are its shortest and its longest codeword? You can use the `camzip` code above changing the method to `'huffman'` to test the compression rate etc. You may also want to do it by hand to test the error resilience:

In [57]:
c = xtree2code(xt)
hamlet_huf = vl_encode(hamlet, c)
hamlet_decoded = vl_decode(hamlet_huf, xt)
print(''.join(hamlet_decoded[:294]))

        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. (KING CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


We now introduce a random bit flip (bit 400 flipped) in the compressed sequence and observe the result.

In [58]:
hamlet_corrupted = hamlet_huf.copy()
hamlet_corrupted[400] ^= 1
hamlet_decoded = vl_decode(hamlet_corrupted, xt)
print(''.join(hamlet_decoded[:297]))

        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. y,; 
aNG CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


## Arithmetic coding

We first try "by hand" to operate the steps of arithmetic coding using floating point numbers. We first compute the cumulative probability distribution.

In [59]:
f = [0.0]
for a in p:
    f.append(f[-1]+p[a])
f.pop()
f = dict([(a,f[k]) for a,k in zip(p,range(len(p)))])

We now perform by hand the first `n=4` steps of arithmetic coding. Vary `n` to observe the loss of precision. 

In [60]:
lo, hi = 0.0, 1.0
n = 4
for k in range(n):
    a = hamlet[k]
    lohi_range = hi - lo
    hi = lo + lohi_range * (f[a] + p[a])
    lo = lo + lohi_range * f[a]
print(f'lo = {lo}, hi = {hi}, hi-lo = {hi-lo}')

lo = 0.03994572862162076, hi = 0.04551184792510039, hi-lo = 0.005566119303479632


The output sequence is roughly the binary expression of `lo` (not exactly) and we can compute and observe it. What length `ell` would we need when encoding all of Hamlet?

In [61]:
from math import floor, ceil
ell = ceil(-log2(hi-lo))+2 if hi-lo > 0.0 else 96
print(bin(floor(lo*2**ell)))

0b101000


We encode and decode Hamlet again using arithmetic coding and verify that the first few lines of the play look as expected.

In [39]:
import arithmetic as arith
arith_encoded = arith.encode(hamlet, p)
arith_decoded = arith.decode(arith_encoded, p, Nin)
print('\n'+''.join(arith_decoded[:294]))

Arithmetic decoded 99%    
        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. (KING CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


VOLTIMAND       |
        |
CORNELIUS       |
        |
ROSENCRANTZ     |  courtiers.
        |
GUILDENSTERN    |
        |
OSRIC   |


        A Gentleman, (Gentlemen:)

        A Priest. (First Priest:)


MARCELLUS       |
        |  officers.
BERNARDO        |


FRANCISCO       a soldier.

REYNALDO        servant to Polonius.
        Players.
        (First Player:)
        (Player King:)
        (Player Queen:)

        Two Clowns, grave-diggers.
        (First Clown:)
        (Second Clown:)

FORTINBRAS      prince of Norway. (PRINCE FORTINBRAS:)

        A Captain.

        English Ambassadors. (First Ambassador:)

GERTRUDE        queen of Denmark, and mother to

We now repeat the steps above but introduce a one bit mistake (bit 399 flipped) and observe the effect on the decoded text. Repeat this experiment varying the location of the mistake or adding more than one mistake. What do you observe? Can you explain why?

In [68]:
arith_corrupted = arith_encoded.copy()
arith_corrupted[399] ^= 1
arith_decoded = arith.decode(arith_corrupted, p, Nin)
print('\n'+''.join(arith_decoded[:294]))

Arithmetic decoded 99%    
        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. I he   soeoW  Llon,o gn swt  s  .em urlo agdehnainf ts 
  aele 
 Nhhyr  a  Sh.ath rue oetantoboso nsiiho- eArpIntn      yd  qtr ayeth soon  he hh? Sesct aT  hLoT e n   Hysst 
o a gw sf  et deLtlucEir on  en  tawfiui 
